In [1]:
# added to remove some of the performance warnings
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

from argparse import ArgumentParser

import numpy as np
from lal import GreenwichMeanSiderealTime
from gwbench import Network

In [2]:
np.set_printoptions(linewidth=200)

parser = ArgumentParser()
parser.add_argument('--derivs', type = str, help = 'Specify which differentiation method to use: [num, sym].', default = 'num')

parser.add_argument("-f", required=False)

_StoreAction(option_strings=['-f'], dest='f', nargs=None, const=None, default=None, type=None, choices=None, required=False, help=None, metavar=None)

In [3]:
# choose between numeric or symbolic derivatives ['num', 'sym']
derivs = parser.parse_args().derivs

# user's choice: waveform to use
wf_model_name = 'lal_bbh'
# wf_other_var_dic = None
wf_other_var_dic = {'approximant':'TaylorF2ecc'}

user_waveform = None

In [4]:
# example detector location defined by user (keep it empty if the detector tech is not custom defined )
user_locs = {}

# example detector psd defined by the user (keep it empty if the detector tech is not custom defined )
user_psds = {}

# user's choice: with respect to which parameters to take derivatives for the Fisher analysis
# deriv_symbs_string = None
deriv_symbs_string = 'Mc eta DL tc phic e0 ra dec psi iota'

# user's choice: convert derivatives to cos or log for specific variables
conv_cos = ()
conv_log = ()

# if numeric  derivatives, user's decision, 1 for True, 0 for False
use_rot = 0

# calculate SNRs, error matrices, and errors only for the network (If 1, will calculate the mentioned values only for network otherwise
# will calculate it for detectors as well as network)
only_net = 1

# number of cores to use for parallelize of the calc_det_responses_derivs
num_cores = None

In [5]:
# options for numeric derivative calculation
# user's choice: switch particular partial derivatives to be analytical, options = [DL,tc,phic,ra,dec,psi]
ana_deriv_symbs_string = None

# choose numdifftools parameters for numerical derivatives
step = 1e-9
method = 'central'
order = 2

# only relevant for symbolic derivatives
gen_derivs = None

In [6]:
# 'user-tec_user-loc' is for custom define detector and its location
# network_spec = ['ET_V']
# network_spec2 = ['CE-40_H']
# network_spec3 = ['CE-40_L']
network_spec = ['ET_V', 'CE-40_H', 'CE-40_L']

print('network spec: ', network_spec)
print()

network spec:  ['ET_V', 'CE-40_H', 'CE-40_L']



In [7]:
import astropy as ap
from gwbench.utils import MTsun
from gwbench.basic_relations import DL_of_z_ap_cosmo, f_isco, f_isco_Msolar, M_of_Mc_eta, Mc_of_M_eta, eta_of_q

# Has to be chosen by user
realizations=100

# Function to generate a random individual masses from a log-uniform distribution
def log_uniform(min_val, max_val, size=realizations):
    """
    Generate a random number from a log-uniform distribution.

    Parameters:
    min_val (float): Minimum value of the distribution.
    max_val (float): Maximum value of the distribution.

    Returns:
    float: Random number from log-uniform distribution.
    """
    return np.exp(np.random.uniform(np.log(min_val), np.log(max_val), size))


# choose the individual masses higher limit acc to the lower sensitivity of the detector
mass1 = log_uniform(5, 400)
mass2 = log_uniform(5, 400)

# Total mass of the binary
total_mass = mass1+mass2

# Calculate the mass ratio
massRatio = np.zeros(len(mass1))
for i in range(len(mass1)):
    if mass1[i] < mass2[i]:
        massRatio[i] = mass2[i] / mass1[i]
    else:
        massRatio[i] = mass1[i] / mass2[i]

# symmetric mass ratio
eta = eta_of_q(massRatio)

# Chirp mass
Mc = Mc_of_M_eta(total_mass, eta)

# Initial ecc
ecc = log_uniform(0.01, 0.2)

# Redshift and Luminosity distance
z = np.random.uniform(0, 20, size=realizations)
DL = DL_of_z_ap_cosmo(z, cosmo=None)

# Sky and polarization angles
cos_iota = np.random.uniform(-1, 1, size=realizations)
cos_dec = np.random.uniform(-1, 1, size=realizations)

iota = np.arccos(cos_iota)
ra = np.random.uniform(0, 2 * np.pi, size=realizations)
dec = np.arccos(cos_dec)
psi = np.random.uniform(0, 2 * np.pi, size=realizations)

In [8]:
import astropy as ap
from gwbench.utils import MTsun
from gwbench.basic_relations import DL_of_z_ap_cosmo, f_isco, f_isco_Msolar, M_of_Mc_eta, Mc_of_M_eta, eta_of_q

In [10]:
# Store SNRs and errors for each set of parameters
snr = []

for i in range(realizations):
    inj_params = {
        'Mc': Mc[0],
        'e0': ecc[i],
        'eta': eta[0],
        'chi1x': 0.,
        'chi1y': 0.,
        'chi1z': 0.,
        'chi2x': 0.,
        'chi2y': 0.,
        'chi2z': 0.,
        'DL': DL[0],
        'tc': 0.,
        'phic': 0.,
        'iota': iota[0],
        'ra': ra[0],
        'dec': dec[0],
        'psi': dec[0],
        'gmst0': 0.,
        'z': z[0],
    }

    
    # choose f_low acc to the detector in the network
    f_hi = f_isco_Msolar(total_mass[0])
    df = 2.**-4

    Network
    f_lo = 3
    f = np.arange(f_lo, f_hi + df, df)

    # Initialize the Network object
    net = Network(network_spec, logger_name='detector', logger_level='INFO')
    

    # Set network variables for the current injection parameters

    net.set_net_vars(
        wf_model_name=wf_model_name,
        wf_other_var_dic=wf_other_var_dic,
        user_waveform=user_waveform,
        f=f,
        inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        conv_cos=conv_cos,
        conv_log=conv_log,
        use_rot=use_rot,
        user_locs=user_locs,
        user_psds=user_psds,
        ana_deriv_symbs_string=ana_deriv_symbs_string
    )

    
    # Extract the snr
    net.calc_snrs()
    snr.append(net.snr)


# print("SNRs for n realizations:")
print(snr)
print(ecc)

2024-08-14 23:20:31,729 - detector - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-14 23:20:31,734 - detector - INFO : Detector responses calculated.
2024-08-14 23:20:31,735 - detector - INFO : SNRs calculated.
2024-08-14 23:20:31,751 - detector - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-14 23:20:31,755 - detector - INFO : Detector responses calculated.
2024-08-14 23:20:31,756 - detector - INFO : SNRs calculated.
2024-08-14 23:20:31,769 - detector - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-14 23:20:31,772 - detector - INFO : Detector responses calculated.
2024-08-14 23:20:31,773 - detector - INFO : SNRs calculated.
2024-08-14 23:20:31,785 - detector - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-14 23:20:31,789 - detector - INFO : Detector responses calculated.
2024-08-14 23:20:31,789 - detector - INFO : SNRs calculated.
2024-08-14 23:20:31,800 - detector - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-14 23:20:31,804 - detec

[11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563, 11.984945314074563,